In [ ]:
import pickle
import os
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import time
import keras
from keras import utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np

In [ ]:
path = '../Local-Final-project/Dataset/alfonso_xii/alfonsoXII_1.jpg'
im = cv2.imread(path)
imres = cv2.resize(im,(360,360))
print(im.shape, imres.shape)
plt.figure()
plt.imshow(imres)

In [ ]:
def resizeDataset(path):
    resizedim = []
    for pc in glob.glob(f"../Local-Final-project/{path}/*/*"):
        resizedim.append(cv2.resize(cv2.imread(pc), (360,360)))
    return np.asarray(resizedim)

In [ ]:
for pc in glob.glob("../Local-Final-project/Dataset/*/*"):
    print(pc)
    break


In [ ]:
import os

path, dirs, files = next(os.walk("../Local-Final-project/Dataset"))
dirs_count = len(dirs)
dirs_count

In [ ]:
def groundTruth(folder):
    path, dirs, files = next(os.walk(f"../Local-Final-project/{folder}"))
    GT=[]
    for pc in glob.glob(f"../Local-Final-project/{folder}/*/*"):
        for i in range(0,len(dirs)):
            if pc.split('/')[-2] == dirs[i]:
                #print(pc.split('/')[-2], i)
                GT.append(i)
    return np.asarray(GT)

In [ ]:
y = groundTruth('Dataset')

In [ ]:
X = resizeDataset('Dataset')

In [ ]:
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
filepath='Checkpoint_{epoch:02d}_{val_accuracy:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]



In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(7, 7),
                 activation='relu',
                 input_shape=(360,360,3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(13, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
batch_size = 25
epochs = 20

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_list)

In [ ]:
moment=time.localtime()
name='Model_{}_{}-{}-{}.h5'.format(model.history.history["val_accuracy"][-1],moment[2],moment[3],moment[4])
model.save(name)

In [ ]:
import json

model_json = model.to_json()
with open(name+'.json', "w") as json_file:
    json.dump(model_json, json_file)

model.save_weights(name)